In [132]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os
from sklearn.cluster import KMeans
import seaborn as sb
from scipy.stats import pearsonr
from sklearn.preprocessing import StandardScaler
import networkx as nx
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from sklearn.linear_model import LogisticRegression
from statsmodels.regression.linear_model import OLS
from statsmodels.tools.tools import add_constant
from statsmodels.discrete.discrete_model import Logit
from statsmodels.iolib.summary2 import summary_col
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , r2_score
from sklearn import linear_model
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

In [ ]:
df = pd.read_excel('cylinder4.xlsx')
df.columns

In [102]:
## rules
df['rules'] = ((df['BCT_Avg'] < df['BCT_Spec']) & ((((df['BCT_Avg'] - df['BCT_Spec'])/df['BCT_Spec'])*100) < 0.03)).astype(int)
df['Finished_Product_Hig'] = (((300<df['Finished_Product_Hig']) & (df['Finished_Product_Hig']<600))).astype(int)
df['Speed_Act_bin'] =(df['Speed_Act']<350).astype(int) 

In [ ]:
df.head()

In [ ]:
df.columns

In [105]:
### TO remove all useless columns 
try:
    df.drop('ï»¿Ã¯Â»Â¿Index',inplace= True , axis =1)
    df.drop('Order_Number',inplace= True , axis =1)

    df.drop('Raw_Mat_Combination',inplace= True , axis =1)
    df.drop('GL',inplace=True,axis=1)
    df.drop('CL',inplace=True,axis=1)
    df.drop('New_Printing_Gap_1',inplace=True,axis=1)
    df.drop('New_Printing_Gap_2',inplace=True,axis=1)
    df.drop('New_Printing_Gap_3',inplace=True,axis=1)
    df.drop('New_Printing_Gap_4',inplace=True,axis=1)
    df.drop('New_Average_Printing_Gap',inplace=True,axis=1)
    df.drop('Number_of_use_Printing_Cylinder',inplace=True,axis=1)
    df.drop('Rwf',inplace=True,axis=1)
    df.columns
except:
    print("already deleted")



In [ ]:
df['rules'].sum()

In [107]:
## Spliting the data
df_true = df.loc[df['rules']==1].copy()
df_true.head(2)
### total true value is 3192
## split the data of false
df_false = df.loc[df['rules']==0].copy()
df_false.head(2)
df_false['rules'].count
### Total value of flase is 30799 prod rejected 
### Note taking randome value from each
df_false_train = df_false.sample(4000)
df_false_train.head(3)
### merge both the data frame
df_train_1 = [df_true,df_false_train]
df_train = pd.concat(df_train_1)

In [ ]:
df_train.describe()
### Droping the column which has - until a solution is found
df_train.columns
### Need to reomve String value else the model will fail  or convert them using any group value
list1 = list(df_train)
for i in list1:
    if df_train[i].dtype == 'object':
        try:
            df_train[i] = df_train[i].astype(float)
        except:
            print(f"unable at columns {i}")

### conver this col to decimal 
df_train['Raw_Mat_Strength_Additive_Percent'] = df_train['Raw_Mat_Strength_Additive_Percent'].apply(lambda x: float(x.replace('%',''))/100 if isinstance(x,str) and '%' in x else x)
df_train['Raw_Mat_Strength_Additive_Percent']

### Dropping additive for now
try:
    df_train.drop('SKU',inplace=True,axis=1)
    df_train.drop('Raw_Mat_Strength_Type',inplace=True,axis=1)
    df_train.drop('Raw_Mat_Strength_Additive',inplace= True, axis=1)
    df_train.drop('BM',inplace=True,axis=1)
    df_train.drop('BL',inplace=True,axis=1)
    df_train.drop('CM',inplace=True,axis=1)
except:
    print("hmm already deleted")


In [ ]:
df_train.columns

In [ ]:
shuffled_indices = np.random.permutation(len(df_train))
test_size_data = int(len(df_train) * 0.2)
test_indices = shuffled_indices[:test_size_data]
train_indices = shuffled_indices[test_size_data:]
df_train.iloc[train_indices]
df_train.iloc[test_indices]
print(len(train_indices)) #### train value split to 80 %
print(len(test_indices)) ##### test value split to 20%

In [ ]:
df_train.columns

In [118]:
y = df_train['rules']
features = ['BCT1', 'BCT2', 'BCT3', 'BCT4', 'BCT5', 'BCT_Avg', 'BCT_Spec', 'ECT1',
       'ECT2', 'ECT3', 'ECT4', 'ECT5', 'ECT_Avg', 'Speed_Act', 'Feed_Roll_Gap',
       'Printing_Cylinder_Gap_1', 'Printing_Cylinder_Gap_2',
       'Printing_Cylinder_Gap_3', 'Printing_Cylinder_Gap_4',
       'GLWeigth(Grammage)', 'BMWeigth(Grammage)', 'BLWeigth(Grammage)',
       'CMWeigth(Grammage)', 'CLWeigth(Grammage)', 'Finished_Product_Wid',
       'Finished_Product_Leg', 'Finished_Product_Hig',
       'Raw_Mat_Strength_Additive_Percent','Speed_Act_bin']
x = df_train[features]

In [122]:
x_train, x_valid , y_train , y_valid = train_test_split(y,x,train_size=0.8,test_size=0.2,random_state=1)
one_hot_code = OneHotEncoder(handle_unknown='ignore')
numerical_imputer = SimpleImputer(strategy='mean')
categorical_imputer = SimpleImputer(strategy='most_frequent')

In [123]:
numerical_transformer = Pipeline(steps=[
  ('imputer', numerical_imputer),
  ('scaler', StandardScaler())
])
# impute and encode
categorical_transformer = Pipeline(steps=[
  ('imputer', categorical_imputer),
  ('onehot', one_hot_code)
])

In [134]:
numerical_cols =  x_train.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = x_train.select_dtypes(include=['object', 'category']).columns

preprocessor = ColumnTransformer(
  transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols),
])

In [ ]:
model = RandomForestRegressor(n_estimators=9, random_state=1)
pipeline = Pipeline(steps=[
  ('preprocessor', preprocessor),
  ('model', model)
])